In [1]:
from netCDF4 import Dataset
import time
import numpy
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num
import pandas as pd

In [3]:
# write data name you want
with Dataset('wins.nc', 'w',format="NETCDF4") as rootgrp:
    #set ur Dimension 
    #lon is 116~128  0.025
    #lat is 18~30    0.025
    time = rootgrp.createDimension("time", None)
    lat = rootgrp.createDimension("lat", 481)
    lon = rootgrp.createDimension("lon", 481) 
    
    #set lon lat
    lonx=np.arange(116,128.01,0.025)
    laty=np.arange(18,30.01,0.025)
    
    #set variable 
    times = rootgrp.createVariable("time","f8",("time",))
    latitudes = rootgrp.createVariable("lat","f4",("lat",))
    longitudes = rootgrp.createVariable("lon","f4",("lon",))
    sst = rootgrp.createVariable("sst","f4",("time","lat","lon"),fill_value=-999.)
    
    #set the info.
    longitudes.axix="X"
    longitudes.units = "degrees east"
    longitudes.point_spacing="even"
    longitudes.modulo=360.
    longitudes.standard_name="longitude"

    latitudes.axix="Y"
    latitudes.units = "degrees north"
    latitudes.point_spacing="even"
    latitudes.standard_name="latitude"
    
    times.units = "hours since 2019-03-06 00:00:00.0"
    times.axis="T"
    times.time_origin="06-MAR-2019"
    times.standard_name="time"
    
    sst.long_name="water_temp"
    sst.missing_value=-999.
    
    #read data
    data=pd.read_csv('OCM.19030600-sst.000',sep='\s+',header=None)
    data2=pd.read_csv('OCM.19030600-sst.001',sep='\s+',header=None)
    longitudes[:]=lonx
    latitudes[:]=laty
    
    sst[0,:,:]=data[2].values.reshape(481,481)
    sst[1,:,:]=data2[2].values.reshape(481,481)
    
    # set the time from to 
    dates = [datetime(2019,3,6)+n*timedelta(days=1) for n in range(sst.shape[0])]
    times[:] = date2num(dates,units=times.units)
    dates = num2date(times[:],units=times.units)